In [ ]:
# Based on AWS Sagemaker Jumpstart notebook from Stable-XL marketplace offer https://aws.amazon.com/marketplace/pp/prodview-pe7wqwehghdtm 
# and my previous post with the original stable diffusion model https://rahulaga.medium.com/from-words-to-pictures-text-to-image-generation-8512b61002fa
# Subscribe to the AWS marketplace offer first. After you accept offer this should work in Sagemaker Studio

In [ ]:
#Setup
!pip install 'stability-sdk[sagemaker] @ git+https://github.com/Stability-AI/stability-sdk.git@sagemaker'
! pip install protobuf==3.20

import sagemaker
from sagemaker import ModelPackage, get_execution_role
from stability_sdk_sagemaker.predictor import StabilityPredictor
from stability_sdk_sagemaker.models import get_model_package_arn
from stability_sdk.api import GenerationRequest, GenerationResponse, TextPrompt

from PIL import Image
from PIL import ImageDraw
from typing import Union, Tuple
import io
import os
import base64
import boto3
from sagemaker.utils import name_from_base

In [ ]:
#model ARNs based on region
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/sdxl-v1-0-2042286-300c57d4fa1e39968d711d754640b0b6"
}

region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")
package_arn = model_package_map[region]  # TODO

role_arn = get_execution_role()
sagemaker_session = sagemaker.Session()

#This will add a model in your SageMaker > Models
model = ModelPackage(role=role_arn,model_package_arn=package_arn,sagemaker_session=sagemaker_session,predictor_cls=StabilityPredictor)

In [ ]:
#Create endpoint
endpoint_name=name_from_base('xl-jumpstart')

# Deploy the ModelPackage. This will take 5-10 minutes to run

instance_type="ml.g5.2xlarge" # valid instance types for this model are ml.g5.2xlarge, p4d.24xlarge, and p4de.24xlarge
deployed_model = model.deploy(initial_instance_count=1,instance_type=instance_type,endpoint_name=endpoint_name)

#Its ready once progress bar ends with !
#--------------!

In [ ]:
#Use with stability predictor
deployed_model = StabilityPredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
#Run inference on this deployment
#The Stability XL has many options but simplifying use similar to my old attempt. 
#This doc has the supported details: https://api.stability.ai/docs#tag/v1generation/operation/textToImage 

In [ ]:
def decode_and_show(model_response: GenerationResponse, prompt) -> None:
    """
    Decodes and displays an image from SDXL output along with prompt used
    """
    image = model_response.artifacts[0].base64
    image_data = base64.b64decode(image.encode())
    image = Image.open(io.BytesIO(image_data))
    
    #add a title of the prompt
    draw = ImageDraw.Draw(image)
    position = (3, 3)
    box = draw.textbbox(position, prompt)
    draw.rectangle(box, fill="white")
    draw.text(position, prompt, fill="black")
    
    display(image)
        
def generate_prompt(prompt, negative_prompts=[]):
    output = deployed_model.predict(
        GenerationRequest(text_prompts=[TextPrompt(text=prompt)], width=1024, height=1024,
                         negative_prompts=negative_prompts))
    decode_and_show(output, prompt)
    
def generate_multi_prompt(prompts, negative_prompts=[]):
    """
    each prompt can be like [TextPrompt(text=prompt, weight=0.5)]
    """
    #combine all prompts into a string for image title label
    title=""
    for prompt in prompts:
        title+=prompt.text + str(prompt.weight)
    
    #generate image
    output = deployed_model.predict(
        GenerationRequest(text_prompts=prompts, width=1024, height=1024,
                         negative_prompts=negative_prompts))
    
    decode_and_show(output, title)

In [ ]:
#Generate multi-prompt images

In [ ]:
generate_multi_prompt(prompts=[TextPrompt(text="one small llama sitting and yawning in the park", weight=4),
                               TextPrompt(text="sunny day in a futuristic san francisco skyline in the background", weight=5),
                               TextPrompt(text="clouds", weight=-1)])

In [ ]:
generate_multi_prompt(prompts=[TextPrompt(text="tall person", weight=1),
                               TextPrompt(text="the person is running around a track", weight=2),
                               TextPrompt(text="the person is holding a flower", weight=1.5)])

In [ ]:
#Generate some samples using same prompts as previously. See prior post: https://rahulaga.medium.com/from-words-to-pictures-text-to-image-generation-8512b61002fa

In [ ]:
generate_prompt("blue stuffy on a tree on a cloudy day with a lot of snow")

In [ ]:
generate_prompt("draw van gogh flowers on a starry night")

In [ ]:
generate_prompt("yop")

In [ ]:
generate_prompt("poop")

In [ ]:
generate_prompt("cats looking realistic playing chess")

In [ ]:
generate_prompt("80000000000000000000000000000000000000000000000000000000000000000000")

In [ ]:
generate_prompt("5876 poops")

In [ ]:
generate_prompt("sunny day, birds chirping, ocean waves, photo realistic")

In [ ]:
generate_prompt("lollipops with a face screaming")

In [ ]:
generate_prompt("psychosis")

In [ ]:
generate_prompt("mental health and generative ai")

In [ ]:
generate_prompt("fractals of colorful eyes")

In [ ]:
generate_prompt("fractals of fingers, greyscale with colorpop")

In [ ]:
#Emotions and gender

In [ ]:
generate_prompt("an angry man pounding his fist and shouting")

In [ ]:
generate_prompt("happy woman walking on the beach with a dog at sunset")

In [ ]:
generate_prompt("an nervous person as a cartoon")

In [ ]:
generate_prompt("a transgender woman waving hand")

In [ ]:
generate_prompt("a transgender man holding a book")

In [ ]:
#Cultural preferences and biases

In [ ]:
generate_prompt("a beautiful woman in 3d model style")

In [ ]:
generate_prompt("a beautiful woman in pixel art style")

In [ ]:
generate_prompt("a beautiful asian woman in cinematic style")

In [ ]:
generate_prompt("a handsome man")

In [ ]:
generate_prompt("Indian wedding")

In [ ]:
generate_prompt("henna on hand with fine detail")

In [ ]:
generate_prompt("quinceañera celebration")

In [ ]:
generate_prompt("doctor reviewing a chart")

In [ ]:
# this will fail - filtered words
generate_prompt("ugly child")

In [ ]:
#Tangible objects

In [ ]:
generate_prompt("cars on the freeway")

In [ ]:
generate_prompt("old rotary phone")

In [ ]:
generate_prompt("boardgame with cards and 2 dice")

In [ ]:
#Intangibles

In [ ]:
generate_prompt("music")

In [ ]:
generate_prompt("patent for generating software")

In [ ]:
generate_prompt("software")

In [ ]:
#Famous persons and locations

In [ ]:
generate_prompt("George Washington outside the White House")

In [ ]:
generate_prompt("Beyonce and Taylor Swift at the Eiffel Tower")

In [ ]:
generate_prompt("Einstein dancing with Cleopatra at the Taj Mahal")

In [ ]:
#Fantasy and fiction

In [ ]:
generate_prompt("terminator fighting with a cow")

In [ ]:
generate_prompt("Superman and Darth Vader fist bump")

In [ ]:
generate_prompt("Mario with Mickey Mouse")

In [ ]:
generate_prompt("Leaning tower of Pisa in times square")

In [ ]:
generate_prompt("Children in the clouds")

In [ ]:
#Spanish

In [ ]:
generate_prompt("estoy durmiendo y soñando con unicornios") 

In [ ]:
generate_prompt("un salón de clases con un escritorio y sillas")

In [ ]:
generate_prompt("fiesta de cumpleaños con pastel y globos") 

In [ ]:
generate_prompt("unattractive tall woman, long hair, brown eyes, photo real", negative_prompts=["feet"])

In [ ]:
generate_prompt("attractive tall man, short hair, black eyes, ultra photo realistic", negative_prompts=["hand"])

In [ ]:
generate_prompt("side view, attractive tall man, grey hair, black eyes, young, ultra photo realistic", negative_prompts=["hand","facial hair"])

In [ ]:
generate_prompt("think of a human face, then draw with four different views of front, back, left side and right side, photo realistic", negative_prompts=["limbs"])

In [ ]:
generate_prompt("an attrative Latina, face with smiling expression. Tinsel in the hair", negative_prompts=["body"])

In [ ]:
generate_prompt("a repulsive looking face", negative_prompts=["body"])

In [ ]:
generate_prompt("create a person with the body of an animal with the face of a human. show the full body and head")

In [ ]:
#Using some ChatGPT generated prompts based on my questions asking for prompt suggestions

In [ ]:
def generate_array(prompts):
    for prompt in prompts:
        generate_prompt(prompt)

In [ ]:
image_generation_prompts = [
    "Generate a portrait of an elderly man with a kind smile and wise eyes.",
    "Create a realistic image of a young woman with curly hair and piercing green eyes.",
    "Generate a portrait of a young man with a rugged look and a slight beard.",
    "Create an image of a middle-aged woman with short hair and a friendly expression.",
    "Generate a portrait of a woman with rosy cheeks and bright eyes, full of wonder and curiosity.",
    "Create an image of a person with a serious expression and strong facial features.",
    "Generate a portrait of a woman with a unique hairstyle and bold makeup.",
    "Create an image of a man with a beard and glasses, looking contemplative.",
    "Generate a portrait of a person with piercing blue eyes and a mysterious expression.",
    "Create an image of a person with a bright smile and sparkling eyes, radiating happiness and joy."
]
generate_array(image_generation_prompts)

In [ ]:
image_generation_prompts = [
    "Imagine you're flying on the back of a giant eagle, soaring over mountains and valleys, feeling the wind rushing through your hair.\n",
    "Picture yourself walking through a forest of glowing mushrooms, the light casting a surreal glow over everything around you.\n",
    "You find yourself in a magical library, where every book holds a world of its own. You can choose any book and step into its pages, living out the story as if it were real.\n",
    "You step through a shimmering portal into a world of floating islands and soaring airships, where the sky is an endless expanse of vibrant colors and you can explore to your heart's content.\n",
    "You're a guest at a royal ball, where you dance the night away with elegant and exotic creatures from all corners of the realm, the music carrying you away into a world of enchantment.\n",
    "You venture deep into a mysterious cavern, where you find a glowing crystal that shows you visions of distant lands and times, revealing secrets of the universe.\n",
    "You awaken in a magical garden, filled with strange and wondrous creatures, each with a unique power or gift to share with you.\n",
    "You find yourself on a vast plain, surrounded by a circle of standing stones, and you suddenly realize that you have the power to control the elements of nature - the wind, the rain, the sun, and the earth.\n",
    "You're on a quest to find a legendary artifact, journeying through dark forests, treacherous mountains, and ancient ruins, facing challenges and meeting allies along the way.\n",
    "You discover a hidden portal that takes you to a realm of dreams, where you can explore the subconscious mind and unlock the secrets of the human psyche.\n"
]
generate_array(image_generation_prompts)

In [ ]:
image_generation_prompts = [
    "Imagine you're standing in the heart of New York City's Times Square, surrounded by the bright lights and buzzing energy of one of the world's most famous destinations. The towering billboards and electronic displays flash advertisements and messages, while street performers and vendors add to the lively atmosphere. You can feel the pulse of the city as people from all over the world rush past you, each on their own mission.",  
    "Picture yourself walking through the ornate halls of Buckingham Palace, home to the British royal family and a symbol of centuries of history and tradition. You can hear the echo of your footsteps on the marble floors as you take in the opulent furnishings and intricate artwork. You might even catch a glimpse of a member of the royal family, as they move through the palace's private chambers.",  
    "You find yourself on the beaches of Rio de Janeiro, where you can dance the samba, play soccer on the sand, and soak up the vibrant culture of one of Brazil's most iconic cities. The warm sand and cool ocean breeze invite you to relax and enjoy the sun, while the sounds of music and laughter fill the air. You can taste the delicious local cuisine and join in the lively conversation with the friendly locals.",  
    "You're sitting in the front row of a concert by your favorite musician, feeling the energy and excitement of the crowd and losing yourself in the music. The lights and sound of the show are intense, and you can feel the bass vibrating through your body. You might even get a chance to meet the artist backstage after the show.",  
    "You're exploring the winding streets and hidden alleyways of Venice, Italy, admiring the historic architecture and charming canals of this timeless and romantic city. You can smell the aroma of freshly baked bread and hear the sound of church bells ringing in the distance. The gondolas and water taxis glide past you on the shimmering canals, adding to the magical atmosphere.",  
    "You're at the top of the Eiffel Tower, looking out over the City of Lights and marveling at the breathtaking views of one of the world's most famous landmarks. The city below spreads out before you like a patchwork quilt, with the Seine River winding through the heart of it. You can see the iconic buildings and monuments of Paris from a unique perspective, and feel the breeze as it rushes by you at this lofty height.",  
    "You find yourself in the heart of Tokyo, surrounded by the neon lights and vibrant culture of one of the world's most bustling and dynamic cities. The streets are packed with people, all moving with purpose and urgency. The food stalls and restaurants offer a dizzying array of options, while the anime shops and karaoke bars are testament to the city's unique blend of old and new.",  
    "You're at a Hollywood movie premiere, walking the red carpet and rubbing shoulders with some of the biggest names in the entertainment industry. The cameras flash as you strike a pose for the paparazzi, and you can hear the murmur of excited fans gathered behind the velvet ropes. The air is electric with anticipation as you make your way into the theater to watch the latest blockbuster film.",  
    "You're standing in front of the Great Pyramid of Giza, one of the most famous and awe-inspiring structures in human history, marveling at the scale and craftsmanship of this ancient wonder. The sun beats down on the sandy landscape as you take in the enormity of the pyramid, which has stood for more than 4,500 years. You can feel the weight of history and culture as you stand in its shadow.",  
    "You're sitting in the stands at the Super Bowl, watching the biggestfootball game of the year with millions of viewers tuning in around the world. The atmosphere is electric as the two teams take to the field, and you can feel the excitement building with every play. The halftime show is a spectacle of music and dance, while the commercials are some of the most talked-about of the year. You can taste the salty snacks and cold drinks as you cheer on your favorite team to victory." ]

generate_array(image_generation_prompts)

In [ ]:
emotions_with_descriptions = [
    "Emotion: Happiness\nFacial Expression: A genuine smile, often with raised cheeks and crinkling at the corners of the eyes (crow's feet).\nDescription: Happiness is characterized by a positive and joyful feeling. The smile is warm and welcoming, and the eyes appear bright and engaged.",
    "Emotion: Sadness\nFacial Expression: Downturned corners of the mouth, drooping eyelids, and a furrowed brow.\nDescription: Sadness reflects a feeling of sorrow or unhappiness. The facial expression may appear dejected and lack the energy associated with other emotions.",
    "Emotion: Anger\nFacial Expression: Furrowed eyebrows, narrowed eyes, tensed jaw, and tightened lips.\nDescription: Anger is marked by a feeling of hostility or frustration. The facial expression is intense, with the focus often on the eyes and mouth.",
    "Emotion: Surprise\nFacial Expression: Widened eyes, raised eyebrows, and an open mouth.\nDescription: Surprise is characterized by a sudden and unexpected reaction to something. The facial expression conveys astonishment and can be accompanied by a quick inhalation.",
    "Emotion: Fear\nFacial Expression: Wide-open eyes, raised eyebrows, open mouth, and tensed facial muscles.\nDescription: Fear is a response to a perceived threat or danger. The facial expression is marked by heightened alertness and readiness for a fight-or-flight response.",
    "Emotion: Disgust\nFacial Expression: Wrinkled nose, raised upper lip, and a squinting or narrowing of the eyes.\nDescription: Disgust is a reaction to something unpleasant or offensive. The facial expression often reflects a sense of aversion.",
    "Emotion: Contempt\nFacial Expression: One corner of the mouth raised in a sneer or smirk, along with a slight raising of the chin.\nDescription: Contempt signifies a feeling of superiority or disdain for someone or something. The facial expression conveys a sense of arrogance.",
    "Emotion: Joy\nFacial Expression: Bright eyes, wide smile, raised cheeks, and an open and relaxed posture.\nDescription: Joy represents a deeper, more intense form of happiness. The facial expression is exuberant and may include laughter or other expressions of delight.",
    "Emotion: Confusion\nFacial Expression: Furrowed brow, tilted head, and a puzzled or uncertain look.\nDescription: Confusion arises when there is uncertainty or lack of understanding. The facial expression often conveys a sense of searching for answers.",
    "Emotion: Contemplation\nFacial Expression: Chin resting on the hand, raised eyebrows, and a thoughtful gaze.\nDescription: Contemplation reflects deep thinking or introspection. The facial expression indicates a person lost in thought."
]

generate_array(emotions_with_descriptions)


In [ ]:
generate_prompt("A pokemon dragon princess")

In [ ]:
#Delete to stop incurring cost
!aws sagemaker list-endpoints
deployed_model.sagemaker_session.delete_endpoint(endpoint_name)

In [ ]:
!aws sagemaker list-endpoints